Ensuring Security of Cloud Server Instances
========================

@DanDye; MIL-OSS Hackathon, Tampa, FL October 23rd - 25th 2015

In [331]:
amis = [ 
{"AMI_ID":"ami-7768011e", "AMI_Name":"RHEL-6.0_GA-i386-6-Hourly2"}, # Done
{"AMI_ID":"ami-196a0370", "AMI_Name":"RHEL-6.1_GA-i386-5-Hourly2"}, # running
{"AMI_ID":"ami-956a03fc", "AMI_Name":"RHEL-6.1_GA-x86_64-5-Hourly2"}, # running
{"AMI_ID":"ami-d96c05b0", "AMI_Name":"RHEL-6.2_GA-i386-4-Hourly2"}, # running
{"AMI_ID":"ami-876c05ee", "AMI_Name":"RHEL-6.2_GA-x86_64-4-Hourly2"}, 
{"AMI_ID":"ami-a35c35ca", "AMI_Name":"RHEL-6.3_GA-i386-5-Hourly2"}, 
{"AMI_ID":"ami-a35a33ca", "AMI_Name":"RHEL-6.3_GA-x86_64-5-Hourly2"},
{"AMI_ID":"ami-7e175617", "AMI_Name":"RHEL-6.4_GA-i386-10-Hourly2"},
{"AMI_ID":"ami-730c631a", "AMI_Name":"RHEL-6.4_GA-i386-7-Hourly2"},
{"AMI_ID":"ami-a25415cb", "AMI_Name":"RHEL-6.4_GA-x86_64-10-Hourly2"},
{"AMI_ID":"ami-7d0c6314", "AMI_Name":"RHEL-6.4_GA-x86_64-7-Hourly2"},
{"AMI_ID":"ami-da3d82b2", "AMI_Name":"RHEL-6.5_GA-20140929-i386-11-Hourly2-GP2"},
{"AMI_ID":"ami-09680160", "AMI_Name":"RHEL-6.0_GA-x86_64-6-Hourly2"},
{"AMI_ID": "ami-1643ff7e", "AMI_Name": "RHEL-6.5_GA-20140929-x86_64-11-Hourly2-GP2"},
{"AMI_ID": "ami-2df5b444", "AMI_Name": "rhel6.4-oracle-ami"},  # done
       ]
AWS_KEYS_MILOSS="AWS_KEYS_MILOSS"
AWS_KEYS_MILOSS_FILE="/home/dandye/.ssh/AWS_KEYS_MILOSS.pem"
AWS_ID = open("/home/dandye/AWS_ID.txt").read().strip()
AWS_KEY = open("/home/dandye/AWS_KEY.txt").read().strip() # keeping secrets isn't easy
AWS_REGION = "us-east-1"

In [332]:
from boto import ec2
for i,ami in enumerate(amis):
    conn = ec2.connect_to_region(AWS_REGION,
                                 aws_access_key_id=AWS_ID,
                                 aws_secret_access_key=AWS_KEY)
    conn.run_instances(ami["AMI_ID"], 
                       key_name="AWS_KEYS_MILOSS",) 
                       #instance_type="t2.micro", # T2 needed for HVM instances
                       #subnet_id="MILOSS")  # VPC needed for T2

In [333]:
# Uses conn from above

filters = {'instance-state-name' : 'running'}
reservations = conn.get_all_instances(filters=filters)
instances = [reservation.instances[0]  for reservation in reservations]

test_amis = {}

for instance in instances:
    print instance
    if "running" in instance.state:
        print instance.image_id
        test_amis[instance.image_id] = {}
        test_amis[instance.image_id].update(instance.__dict__)
    else:
        print instance.state
   
print [test_amis[akey]["_state"] for akey in test_amis.keys()]# FixMe: loop until all states are 'running'

Instance:i-ea762655
ami-da3d82b2
Instance:i-ad772712
ami-a25415cb
Instance:i-737626cc
ami-1643ff7e
[running(16), running(16), running(16)]


In [348]:
"""Setup some SSH/SFTP connex"""
import paramiko

import paramiko

mykey = paramiko.RSAKey.from_private_key_file("/home/dandye/.ssh/AWS_KEYS_MILOSS.pem")
sshusers = ['wontwork', 'ec2-user', 'root']

clients = []
for akey in test_amis.keys():
    public_dns_name = test_amis[akey]["public_dns_name"]
    print public_dns_name
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy()) 
    client.load_system_host_keys()
    for sshuser in sshusers:
        try:
            client.connect(public_dns_name, username=sshuser, pkey=mykey)
            print client
            break # If connect works, no need to try next username
        except:
            print "Username {} failed. Trying next username".format(sshuser)
    clients.append(client)

print clients
#sftp = paramiko.SFTPClient.from_transport(client)

"""Use Paramiko to run remote commands on Rackspace/EC2 Instance"""
def run(client, cmd):
    transport = client.get_transport()
    session = transport.open_session()
    session.set_combine_stderr(True)
    session.get_pty()
    session.exec_command("sudo {}".format(cmd))
    stdin = session.makefile('wb', -1)
    stdout = session.makefile('rb', -1)
    for line in stdout.read().splitlines():        
        print line

ec2-54-204-209-250.compute-1.amazonaws.com
Username wontwork failed. Trying next username
ec2-50-16-87-148.compute-1.amazonaws.com
Username wontwork failed. Trying next username
ec2-174-129-125-33.compute-1.amazonaws.com
Username wontwork failed. Trying next username
[<paramiko.client.SSHClient object at 0x7f4aa4511d90>, <paramiko.client.SSHClient object at 0x7f4aa4511c90>, <paramiko.client.SSHClient object at 0x7f4aa64bb950>]


In [ ]:
# Install pre-reqs and generate the report

profiles = (
    "CS2",
    "common",
    "server",
    "stig-rhel6-server-upstream",
    "usgcb-rhel6-server",
    "rht-ccp",
    "CSCF-RHEL6-MLS",)

for client in clients: # Future work: pool for ||
    run(client, "sudo yum install -y openscap-utils scap-security-guide")
    run(client, "rpm -ql scap-security-guide")
    for profile in profiles:
        print "Executing profile: {}".format(profile)
        run(client, """oscap xccdf eval \
          --profile {profile} \
          --results /tmp/{profile}.xml \
          --report /tmp/{profile}.html \
          --cpe /usr/share/xml/scap/ssg/content/ssg-rhel6-cpe-dictionary.xml \
                /usr/share/xml/scap/ssg/content/ssg-rhel6-xccdf.xml""".format(
            profile=profile)
           ) # FixMe: use fabric to run command on each host

Loaded plugins: amazon-id, rhui-lb, security

rhui-REGION-client-config-server-6                       | 2.9 kB     00:00     

rhui-REGION-client-config-server-6/primary_db            | 6.0 kB     00:00     

rhui-REGION-rhel-server-releases                         | 3.7 kB     00:00     
rhui-REGION-rhel-server-r 20% [===            ]  0.0 B/s | 7.1 MB     --:-- ETA 
rhui-REGION-rhel-server-r 37% [=====-         ]  10 MB/s |  13 MB     00:02 ETA 
rhui-REGION-rhel-server-r 53% [========       ]  11 MB/s |  19 MB     00:01 ETA 
rhui-REGION-rhel-server-r 72% [==========-    ]  11 MB/s |  26 MB     00:00 ETA 
rhui-REGION-rhel-server-r 83% [============-  ]  11 MB/s |  30 MB     00:00 ETA 

rhui-REGION-rhel-server-releases/primary_db              |  36 MB     00:02     

rhui-REGION-rhel-server-releases-optional                | 3.5 kB     00:00     

rhui-REGION-rhel-server-releases-optional/primary_db     | 3.4 MB     00:00     

rhui-REGION-rhel-server-rh-common                        

In [288]:
#from fabric.api import env, run, get
#report = get("/tmp/ssg-results.html", "ami_{}_ssg_results.html".format(AMI_ID))
test_amis.keys()


[u'ami-1643ff7e', u'ami-09680160']

In [286]:
# SCP the report

for ami in test_amis.keys():
    print "Getting reports from ami: {}".format(ami)
    transport = paramiko.Transport((test_amis[ami]["public_dns_name"], 22))
    transport.connect(username=test_amis[ami]["username"], pkey=mykey)
    sftp = paramiko.SFTPClient.from_transport(transport)
    
    for profile in profiles:
        local_name = "{}_{}".format(ami, profile)
        sftp.get("/tmp/{}.html".format(profile), "./reports/{}.html".format(local_name))
        sftp.get("/tmp/{}.xml".format(profile), "./reports/{}.xml".format(local_name))
        test_amis[ami]["local_name"] = local_name

ami-1643ff7e
ami-09680160


AuthenticationException: Authentication failed.

In [228]:
import os
from IPython.display import IFrame
#render = lambda url: IFrame(url, width=900, height=500)

for ami in test_amis.keys():
    url = "{}.html".format(test_amis[ami]["local_name"])
    print url
    IFrame(url, width=900, height=500)
    break

ami-2df5b444_ssg_results.html


In [229]:
IFrame("ami-2df5b444_ssg_results.html", width=900, height=500)

In [328]:
import os
import glob
from IPython.display import HTML, IFrame

base_name = lambda apath: os.path.splitext(os.path.split(apath)[-1])[0]

report_list = []
for report in glob.glob("./reports/*.html"):
    xml = report[:-5]+".xml"
    report_list.append("""<tr>
      <td>Amazon EC2</td>
      <td>
       <a href="{xml}" target="iframe_a">{xml_name}</a>
      </td>
      <td>
       <a href="{report}" target="iframe_a">{report_name}</a>
      </td>
      </tr>""".format(report=report, 
                      report_name=base_name(report),
                      xml_name=base_name(xml),
                      xml=xml))

HTML("""
<head>
<style>
table, th, td {
    border: 1px solid black;
    border-collapse: collapse;
}
th, td {
    padding: 5px;
    text-align: center;
}
</style>
</head>
<table>
     <tr>
      <th>Cloud Provider</th>
      <th>XML Report</th>
      <th>HTML Report</th>
     </tr>

""" + "\n".join(report_list) + """

</table>
<br/>
<iframe src="reports/ami-2df5b444.html" width=800 height=900 name="iframe_a"></iframe>
""")


#IFrame("./reports/ami-2df5b444.html", width=900, height=500)

Cloud Provider,XML Report,HTML Report
Amazon EC2,ami_ami-94a3eefc_ssg_results,ami_ami-94a3eefc_ssg_results
Amazon EC2,ami-2df5b444_rht-ccp,ami-2df5b444_rht-ccp
Amazon EC2,ami-1643ff7e_common,ami-1643ff7e_common
Amazon EC2,ami-1643ff7e_CSCF-RHEL6-MLS,ami-1643ff7e_CSCF-RHEL6-MLS
Amazon EC2,ami-1643ff7e_usgcb-rhel6-server,ami-1643ff7e_usgcb-rhel6-server
Amazon EC2,ami-2df5b444_server,ami-2df5b444_server
Amazon EC2,ami-2df5b444,ami-2df5b444
Amazon EC2,ami-2df5b444_usgcb-rhel6-server,ami-2df5b444_usgcb-rhel6-server
Amazon EC2,ami-2df5b444_CS2,ami-2df5b444_CS2
Amazon EC2,ami-2df5b444_ssg_results,ami-2df5b444_ssg_results
